<a href="https://colab.research.google.com/github/kridtapon/WFO-Few-Number-Connors-RSI-Trend-Rider/blob/main/WFO_Few_Number_Connors_RSI_Trend_Rider.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.5/527.5 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.2 MB/s eta 0:00:00


In [9]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt
import itertools
from scipy.stats import rankdata

# Function to calculate EMA
def calculate_ema(series, period):
    return series.ewm(span=period, adjust=False).mean()

# Function to calculate RSI
def calculate_rsi(series, period=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

# Function to calculate RSI streak
def calculate_rsi_streak(series):
    streak = series.diff().apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
    streak_rsi = calculate_rsi(streak, period=3)
    return streak_rsi

# Function to calculate percentage rank
def calculate_percent_rank(series, period=100):
    return series.rolling(period).apply(lambda x: rankdata(x)[-1] / period * 100, raw=False)

# Function to calculate Connors RSI
def calculate_connors_rsi(df, price_rsi_period=14, streak_rsi_period=3, pct_rank_period=100):
    price_rsi = calculate_rsi(df['Close'], price_rsi_period)
    streak_rsi = calculate_rsi_streak(df['Close'])
    pct_rank = calculate_percent_rank(df['Close'].diff().abs(), pct_rank_period)
    connors_rsi = (price_rsi + streak_rsi + pct_rank) / 3
    return connors_rsi

# Define the stock symbol and time period
symbol = 'VST' # VST
start_date = '2015-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Walk-forward optimization with Connors RSI and Entry/Exit thresholds
def walk_forward_optimization_with_thresholds(df, start_year, end_year):
    results = []

    # Define dynamic ranges for parameters
    ema_period_range = [10, 20, 30, 50, 100]  # Range for EMA periods
    connors_rsi_price_period_range = [3, 5, 10, 14]  # Range for Connors RSI price periods
    connors_rsi_streak_period_range = [3, 5, 7]  # Range for Connors RSI streak periods
    connors_rsi_pct_rank_period_range = [50, 100, 150]  # Range for Connors RSI pct rank periods
    entry_threshold_range = [10, 20, 30]  # Thresholds for entry Connors RSI
    exit_threshold_range = [70, 80, 90]   # Thresholds for exit Connors RSI

    for test_year in range(start_year + 4, end_year + 1):
        train_start = test_year - 4
        train_end = test_year - 1
        test_start = test_year

        train_data = df[(df.index.year >= train_start) & (df.index.year <= train_end)]
        test_data = df[df.index.year == test_year]

        best_params = None
        best_performance = -np.inf

        # Loop through all combinations of parameters
        for params in itertools.product(ema_period_range,
                                        connors_rsi_price_period_range,
                                        connors_rsi_streak_period_range,
                                        connors_rsi_pct_rank_period_range,
                                        entry_threshold_range,
                                        exit_threshold_range):
            ema_period, price_rsi_period, streak_rsi_period, pct_rank_period, entry_threshold, exit_threshold = params

            # Calculate EMA and Connors RSI on the training data
            train_data['EMA'] = calculate_ema(train_data['Close'], ema_period)
            train_data['Connors_RSI'] = calculate_connors_rsi(
                train_data,
                price_rsi_period=price_rsi_period,
                streak_rsi_period=streak_rsi_period,
                pct_rank_period=pct_rank_period
            )

            # Generate entry and exit signals based on Connors RSI and thresholds
            entries = (train_data['Connors_RSI'] < entry_threshold)
            exits = (train_data['Connors_RSI'] > exit_threshold)

            # Backtest on training data
            portfolio = vbt.Portfolio.from_signals(
                close=train_data['Close'],
                entries=entries,
                exits=exits,
                init_cash=100_000,
                fees=0.001
            )

            performance = portfolio.total_return()
            if performance > best_performance:
                best_performance = performance
                best_params = params

        # Test with the best parameters on the test data
        test_data['EMA'] = calculate_ema(test_data['Close'], best_params[0])
        test_data['Connors_RSI'] = calculate_connors_rsi(
            test_data,
            price_rsi_period=best_params[1],
            streak_rsi_period=best_params[2],
            pct_rank_period=best_params[3]
        )

        entries = (test_data['Connors_RSI'] < best_params[4])
        exits = (test_data['Connors_RSI'] > best_params[5])

        portfolio = vbt.Portfolio.from_signals(
            close=test_data['Close'],
            entries=entries,
            exits=exits,
            init_cash=100_000,
            fees=0.001
        )

        results.append({
            'Year': test_year,
            'Best_Params': best_params
        })

    return pd.DataFrame(results)

# Perform walk-forward optimization
results = walk_forward_optimization_with_thresholds(df, 2016, 2025)

# Display results
print("\nWalk-Forward Optimization Results:")
print(results)

# Combine signals into a single portfolio
combined_entries = pd.Series(False, index=df.index)
combined_exits = pd.Series(False, index=df.index)

for _, row in results.iterrows():
    year = row['Year']
    params = row['Best_Params']
    yearly_data = df[df.index.year == year]

    # Apply EMA and Connors RSI indicators
    yearly_data['EMA'] = calculate_ema(yearly_data['Close'], params[0])
    yearly_data['Connors_RSI'] = calculate_connors_rsi(
        yearly_data,
        price_rsi_period=params[1],
        streak_rsi_period=params[2],
        pct_rank_period=params[3]
    )

    # Define entry/exit conditions
    entries = (yearly_data['Connors_RSI'] < params[4])
    exits = (yearly_data['Connors_RSI'] > params[5])

    combined_entries.loc[entries.index] = entries
    combined_exits.loc[exits.index] = exits

# Filter data for testing period only
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]
combined_entries = combined_entries[(combined_entries.index.year >= 2020) & (combined_entries.index.year <= 2025)]
combined_exits = combined_exits[(combined_exits.index.year >= 2020) & (combined_exits.index.year <= 2025)]

# Backtest using the combined signals
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=combined_entries,
    exits=combined_exits,
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()

Streaming output truncated to the last 5000 lines.
<ipython-input-9-d398309f6299>:81: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-9-d398309f6299>:82: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-9-d398309f6299>:81: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return


Walk-Forward Optimization Results:
   Year               Best_Params
0  2020  (10, 10, 3, 100, 30, 90)
1  2021    (10, 5, 3, 50, 30, 80)
2  2022    (10, 5, 3, 50, 30, 80)
3  2023    (10, 5, 3, 50, 30, 80)
4  2024    (10, 5, 3, 50, 30, 90)
5  2025   (10, 5, 3, 100, 30, 90)


<ipython-input-9-d398309f6299>:152: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           586020.727742
Total Return [%]                       486.020728
Benchmark Return [%]                   587.968758
Max Gross Exposure [%]                      100.0
Total Fees Paid                       5813.478033
Max Drawdown [%]                        34.859264
Max Drawdown Duration                       510.0
Total Trades                                   18
Total Closed Trades                            17
Total Open Trades                               1
Open Trade PnL                      -70831.666097
Win Rate [%]                            88.235294
Best Trade [%]                         118.419066
Worst Trade [%]                         -8.164887
Avg Winning Trade [%]                   17.907663
Avg Losing Trade [%]                    -8.023604


In [10]:
# Filter Test Years
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Buy and Hold Performance Metrics
df_holding = df['Close']
pf = vbt.Portfolio.from_holding(df_holding, init_cash=100_000)
pf.stats()

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



,Close
Start,2020-01-02 00:00:00
End,2024-12-31 00:00:00
Period,1258
Start Value,100000.0
End Value,687968.757793
Total Return [%],587.968758
Benchmark Return [%],587.968758
Max Gross Exposure [%],100.0
Total Fees Paid,0.0
Max Drawdown [%],45.749981
